In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
# Chemin vers le fichier CSV
filename = "nouveau_fichier.csv"

# Charger le fichier CSV dans un dataframe pandas
df = pd.read_csv(filename)

# Supprimer les 23 dernières lignes
df = df.iloc[:-24]

# Écrire les données modifiées dans le fichier CSV
df.to_csv("newGPL18545_humanexome-12v1-1_a.csv", index=False)

/tmp/ipykernel_4443/4125819099.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [2]:
with open('GSE148375_processed_data.txt') as f:
    # Lecture de la première ligne pour récupérer les noms des colonnes
    colonnes = f.readline().strip().split('\t')
    
    # Renommer le ID_REF par exm#
    colonnes[0]="exm#"
    
    # Création d'une nouvelle chaîne de caractères pour la première ligne modifiée
    new_first_line = '\t'.join(colonnes) + '\n'
    
    # Écriture de la première ligne modifiée dans un nouveau fichier
    with open('newGSE148375_processed_data.txt', 'w') as fw:
        fw.write(new_first_line)

In [3]:
from pyspark.sql import SparkSession

In [4]:
# Création d'une session Spark
spark = SparkSession.builder.appName("fusion_data").getOrCreate()

# Chargement du fichier GPL18545_humanexome-12v1-1_a.csv dans un DataFrame
df_gpl = spark.read.csv("newGPL18545_humanexome-12v1-1_a.csv", sep=",",header=True, inferSchema=True)

# Sélection des colonnes nécessaires
df_gpl_select = df_gpl.select("IlmnID", "SNP", "Chr", "MapInfo", "RefStrand")

# Chargement du fichier GSE148375_processed_data.txt dans un DataFrame
df_gse = spark.read.csv("GSE148375_processed_data.txt", sep="\t", header=True, inferSchema=True)

23/04/09 15:39:03 WARN Utils: Your hostname, legrand-Aspire-A315-54K resolves to a loopback address: 127.0.1.1; using 192.168.58.128 instead (on interface wlp2s0)
23/04/09 15:39:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 15:39:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df_gpl_select.show(5)

+--------------------+-----+---+------------+---------+
|              IlmnID|  SNP|Chr|     MapInfo|RefStrand|
+--------------------+-----+---+------------+---------+
|exm-IND1-20044998...|[D/I]|  1|2.02183358E8|        -|
|exm-IND1-20145348...|[D/I]|  1|2.03186865E8|        -|
|exm-IND1-85310248...|[I/D]|  1| 8.5537661E7|        +|
|exm-IND10-1028177...|[I/D]| 10|1.02827758E8|        +|
|exm-IND10-1832963...|[D/I]| 10| 1.8289634E7|        -|
+--------------------+-----+---+------------+---------+
only showing top 5 rows



In [6]:
#renommer les noms des colonnes
df_gpl_select = df_gpl_select.withColumnRenamed("IlmnID", "exm#") \
                             .withColumnRenamed("SNP", "alleles") \
                             .withColumnRenamed("Chr", "chrom") \
                             .withColumnRenamed("MapInfo", "pos") \
                             .withColumnRenamed("RefStrand", "strand")

In [7]:
df_gse_select = df_gse.withColumnRenamed("ID_REF", "exm#")

In [8]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def remove_ends(input_string):
    if len(input_string) < 3:
        return "input_string"
    else:
        return input_string[1:-1]

# Créer une fonction UDF pour appliquer la fonction remove_ends à une colonne
udf_remove_ends = udf(remove_ends, StringType())

def remove_ends_column(df, column_name):
    # Ajouter une nouvelle colonne avec les valeurs modifiées
    df = df.withColumn(column_name+"_new", udf_remove_ends(df[column_name]))
    # Remplacer la colonne existante par la nouvelle colonne
    df = df.drop(column_name).withColumnRenamed(column_name+"_new", column_name)
    return df


In [9]:
df_gpl_select=remove_ends_column(df_gpl_select, "alleles")

In [10]:
from pyspark.sql.functions import lit

# Ajout de nouvelles colonnes avec des valeurs "NA"
df_gpl_select = df_gpl_select.withColumn("assembly#", lit("NA")) \
                             .withColumn("center", lit("NA")) \
                             .withColumn("protLSID", lit("NA")) \
                             .withColumn("assayLSID", lit("NA")) \
                             .withColumn("panelLSID", lit("NA")) \
                             .withColumn("QCcode", lit("NA"))

In [11]:
df_gpl_select.count()

242901

In [12]:
df_gpl_select.show(5)

+--------------------+-----+------------+------+-------+---------+------+--------+---------+---------+------+
|                exm#|chrom|         pos|strand|alleles|assembly#|center|protLSID|assayLSID|panelLSID|QCcode|
+--------------------+-----+------------+------+-------+---------+------+--------+---------+---------+------+
|exm-IND1-20044998...|    1|2.02183358E8|     -|    D/I|       NA|    NA|      NA|       NA|       NA|    NA|
|exm-IND1-20145348...|    1|2.03186865E8|     -|    D/I|       NA|    NA|      NA|       NA|       NA|    NA|
|exm-IND1-85310248...|    1| 8.5537661E7|     +|    I/D|       NA|    NA|      NA|       NA|       NA|    NA|
|exm-IND10-1028177...|   10|1.02827758E8|     +|    I/D|       NA|    NA|      NA|       NA|       NA|    NA|
|exm-IND10-1832963...|   10| 1.8289634E7|     -|    D/I|       NA|    NA|      NA|       NA|       NA|    NA|
+--------------------+-----+------------+------+-------+---------+------+--------+---------+---------+------+
only showi

In [13]:
# Jointure des deux DataFrames sur la colonne "IlmnID"
df_join = df_gpl_select.join(df_gse_select, on="exm#")

In [14]:
df_join.count()

In [77]:
# Fusionner toutes les partitions en une seule
df_join = df_join.coalesce(1)

# Enregistrer les données dans un seul fichier CSV
df_join.write.format("csv").option("header", True).option("delimiter", "\t").save("hapmap.csv")

23/04/09 02:44:39 WARN DAGScheduler: Broadcasting large task binary with size 1363.0 KiB
